In [1]:
import random

from const import *
from dataProcess import *
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense



def buildModel(loadParamFromFile = False):
    model = Sequential()
    model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))
    for i in range(DEPTH):
        model.add(LSTM(8, return_sequences=True))
    model.add(LSTM(2, return_sequences=True))
    model.summary()
    model.compile(optimizer='rmsprop',loss='mse')
    if loadParamFromFile:
        model.load_weights(MODELFILE)
    return model


def composeRap(bars, rhymes, model):
    originalBars = getAllBars(LYRICSFILE)
    startingIdx = random.randint(0, len(originalBars)-2)
    # Randomly pick two lines from original lyrics as starting point
    startingLines = originalBars[startingIdx:startingIdx+2]

    startingVector = []
    for bar in startingLines:
        startingVector.append((syllables(bar), ryhmeIndex(bar, rhymes)))
    # Generate True starting point
    startingVector = model.predict(np.array([startingVector]).flatten().reshape(1,2,2))
    rapVector = [startingVector]
    for i in range(100):
        rapVector.append(model.predict(np.array([rapVector[-1]]).flatten().reshape(1,2,2)))

    return rapVector


def calculatePenalty(bars, standardLine, penaltyIndex=0.2):
    # calculate the penalty score last word of all bars deviate from standline
    penalty = 0
    standardWord = standardLine.strip('?!,.').split(" ")[-1]
    for bar in bars:
        lastWord = bar.strip('?!,.').split(" ")[-1]
        if lastWord == standardWord:
            penalty += penaltyIndex
    return penalty


def calculateScore(vector, syllables, rhyme, rhymeListLen, penalty):
    if not rhyme:
        rhyme = 0.0
    desiredSyllables = float(vector[0]) * MAXSYLLABLES
    desiredRhyme = float(vector[1]) * rhymeListLen
    return 1.0 - (abs(desiredSyllables - float(syllables)) +
                  abs(desiredRhyme - float(rhyme))) - penalty


def convertVectorToSong(rapVector, generatedLyrics, rhymes):
    generatedData = []
    rhymeListLen = len(rhymes)
    for bar in generatedLyrics:
        generatedData.append([bar, syllables(bar), ryhmeIndex(bar, rhymes)])

    rap = []
    unfoldVector = []
    for vector in rapVector:
        unfoldVector.append(list(vector[0][0]))
        unfoldVector.append(list(vector[0][1]))

    for vector in unfoldVector:
        scoreList = []
        for bar, syllable, rhymeindex in generatedData:
            if len(rap) != 0:
                penalty = calculatePenalty(rap, bar)
            else:
                penalty = 0
            totalScore = calculateScore(vector, syllable, rhymeindex, rhymeListLen, penalty)
            scoreList.append([bar, totalScore])


        maxScore = max([float(score[1]) for score in scoreList])
        for bar, score in scoreList:
            if score == maxScore:
                rap.append(bar)
                for data in generatedData:
                    if bar == data[0]:
                        generatedData.remove(data)
                        break
                break
    return rap


def train(x_data, y_data, model):
    model.fit(np.array(x_data), np.array(y_data),
              batch_size=2,
              epochs=10,
              verbose=1)
    model.save_weights(MODELFILE)


Using TensorFlow backend.


In [2]:
model = buildModel(loadParamFromFile=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2, 4)              112       
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 8)              416       
_________________________________________________________________
lstm_3 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_4 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_5 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_6 (LSTM)                (None, 2, 2)              88        
Total params: 2,248
Trainable params: 2,248
Non-trainable params: 0
_________________________________________________________________


In [3]:
bars = getAllBars(LYRICSFILE)
rhymes = getAllRhyme(bars, readFromFile=True)

In [4]:
xData, yData = buildDataSet(bars, rhymes)

In [5]:
xData

array([[[ 0.        ,  0.63179916],
        [ 0.        ,  0.16317992]],

       [[ 0.        ,  0.77405858],
        [ 0.        ,  0.29707113]],

       [[ 0.        ,  0.16317992],
        [ 0.        ,  0.82008368]],

       ..., 
       [[ 1.        ,  0.53138075],
        [ 0.        ,  0.17573222]],

       [[ 0.        ,  0.50627615],
        [ 0.        ,  0.90376569]],

       [[ 0.        ,  0.17573222],
        [ 0.        ,  0.17573222]]])

In [10]:
yData

array([[[ 0.        ,  0.77405858],
        [ 0.        ,  0.29707113]],

       [[ 0.        ,  0.16317992],
        [ 0.        ,  0.82008368]],

       [[ 0.        ,  0.29707113],
        [ 0.        ,  0.82008368]],

       ..., 
       [[ 0.        ,  0.50627615],
        [ 0.        ,  0.90376569]],

       [[ 0.        ,  0.17573222],
        [ 0.        ,  0.17573222]],

       [[ 0.        ,  0.90376569],
        [ 0.        ,  0.87029289]]])

In [21]:
type(xData[0][0][0])

numpy.float64

In [31]:
def build_dataset(lines, rhyme_list):
	dataset = []
	line_list = []
	for line in lines:
		line_list = [line, syllables(line), ryhmeIndex(line, rhyme_list)]
		dataset.append(line_list)
	
	x_data = []
	y_data = []
	
	for i in range(len(dataset) - 3):
		line1 = dataset[i    ][1:]
		line2 = dataset[i + 1][1:]
		line3 = dataset[i + 2][1:]
		line4 = dataset[i + 3][1:]

		x = [line1[0], line1[1], line2[0], line2[1]]
		x = np.array(x)
		x = x.reshape(2,2)
		x_data.append(x)

		y = [line3[0], line3[1], line4[0], line4[1]]
		y = np.array(y)
		y = y.reshape(2,2)
		y_data.append(y)
		
	x_data = np.array(x_data)
	y_data = np.array(y_data)
	
	#print "x shape " + str(x_data.shape)
	#print "y shape " + str(y_data.shape)
	return x_data, y_data

In [22]:
def train(x_data, y_data, model):
    model.fit(x_data, y_data,
              batch_size=2,
              epochs=5,
              verbose=1)
    model.save_weights(MODELFILE)

In [6]:
train(xData, yData, model)


Epoch 1/10
6188/6188 [==============================] - 28s 5ms/step - loss: 0.0624
Epoch 2/10
6188/6188 [==============================] - 23s 4ms/step - loss: 0.0551
Epoch 3/10
6188/6188 [==============================] - 23s 4ms/step - loss: 0.0545
Epoch 4/10
6188/6188 [==============================] - 24s 4ms/step - loss: 0.0544
Epoch 5/10
6188/6188 [==============================] - 24s 4ms/step - loss: 0.0544
Epoch 6/10
6188/6188 [==============================] - 25s 4ms/step - loss: 0.0544
Epoch 7/10
6188/6188 [==============================] - 25s 4ms/step - loss: 0.0544
Epoch 8/10
6188/6188 [==============================] - 25s 4ms/step - loss: 0.0544
Epoch 9/10
6188/6188 [==============================] - 24s 4ms/step - loss: 0.0544
Epoch 10/10
6188/6188 [==============================] - 24s 4ms/step - loss: 0.0545


In [36]:
syllables("Most rappers' taste level ain't at my waist level")ryhmeIndex(bar, rhymes)

0

In [37]:
ryhmeIndex("Most rappers' taste level ain't at my waist level", rhymes)

0.41841004184100417

In [38]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))
	model.add(LSTM(2, return_sequences=True))
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')

	return model

In [40]:
m = create_network(4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 2, 4)              112       
_________________________________________________________________
lstm_8 (LSTM)                (None, 2, 8)              416       
_________________________________________________________________
lstm_9 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_10 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_11 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_12 (LSTM)               (None, 2, 2)              88        
Total params: 2,248
Trainable params: 2,248
Non-trainable params: 0
_________________________________________________________________


In [41]:
train(xData, yData, m)

Epoch 1/5
6188/6188 [==============================] - 29s 5ms/step - loss: nan
Epoch 2/5
6188/6188 [==============================] - 24s 4ms/step - loss: nan
Epoch 3/5
6188/6188 [==============================] - 23s 4ms/step - loss: nan
Epoch 4/5
6188/6188 [==============================] - 26s 4ms/step - loss: nan
Epoch 5/5
6188/6188 [==============================] - 26s 4ms/step - loss: nan
